#### Задание на ПР-9:
1. Выбрать на kaggle.com любой датасет с неразмеченными данными (или просто убрать метку). Признаков должно быть не меньше 6, а экземпляров - не меньше 200. Присоединенный винный датасет подойдет.
2. Перевести все значения признаков в числа и шкалировать.
3. Произвести кластеризацию по методу k средних на оптимальное для этого датасета число кластеров. Обосновать выбор оптимального числа кластеров.
4. Произвести восходящую кластеризацию. (На защите буду спрашивать, в частности, что является критерием остановки алгоритма восходящей кластеризации)
5. Сравнить результаты кластеризации, полученные разными методами. Требуется количественное сравнение!
6. Опишите полученные результаты с точки зрения практического применения.

**На этапе загрузки данных исключаем quality и ID.**

Вообще говоря, **ID** исключается практически всегда, так как не несет никакой смысловой нагрузки. 

А что касается **quality**, если бы мы его использовали, то это было бы обучение с учителем, а не кластеризация. То есть мы бы решали задачу не разложения имеющихся данных в виде точек на кластеры, а задачу классификации. Исходя из этого, для задач кластеризации нам необходим неразмеченный датасет.

Цель кластеризации - найти естественные группы, а не предсказать известную метку.

In [1]:
# Импорт библиотек
import math
import random
import collections

In [2]:
# Функции для работы с данными
def load_data(filename):
    # Функция для загрузки данных из CSV файла
    data = []  # Список для хранения данных
    with open(filename, 'r') as file:  # Открываем файл для чтения
        lines = file.readlines()  # Читаем все строки файла
    headers = lines[0].strip().split(',')  # Получаем заголовки столбцов
    for i in range(1, len(lines)):  # Обрабатываем каждую строку данных
        line = lines[i].strip()  # Убираем лишние пробелы
        if line:  # Если строка не пустая
            values = line.split(',')  # Разделяем строку по запятым
            if len(values) == len(headers):  # Проверяем корректность данных
                row = []  # Создаем список для строки данных
                for j in range(len(values) - 2):  # Исключаем quality и Id
                    row.append(float(values[j]))  # Преобразуем в числа
                data.append(row)  # Добавляем строку в данные
    return data, headers[:-2]  # Возвращаем данные и заголовки

def scale_data(data):
    # Функция для масштабирования данных (z-score нормализация)
    if len(data) == 0:  # Проверяем что данные не пустые
        return data
    
    num_features = len(data[0])  # Количество признаков
    scaled_data = []  # Список для масштабированных данных
    
    # Вычисляем средние значения для каждого признака
    means = []  # Список для средних значений
    for j in range(num_features):  # Для каждого признака
        sum_val = 0.0  # Сумма значений признака
        for i in range(len(data)):  # Для каждой строки данных
            sum_val = sum_val + data[i][j]  # Суммируем значения
        mean_val = sum_val / len(data)  # Вычисляем среднее
        means.append(mean_val)  # Добавляем в список средних
    
    # Вычисляем стандартные отклонения для каждого признака
    stds = []  # Список для стандартных отклонений
    for j in range(num_features):  # Для каждого признака
        sum_sq = 0.0  # Сумма квадратов отклонений
        for i in range(len(data)):  # Для каждой строки данных
            diff = data[i][j] - means[j]  # Отклонение от среднего
            sum_sq = sum_sq + diff * diff  # Суммируем квадраты отклонений
        variance = sum_sq / len(data)  # Вычисляем дисперсию
        std_val = math.sqrt(variance)  # Вычисляем стандартное отклонение
        stds.append(std_val)  # Добавляем в список отклонений
    
    # Масштабируем данные
    for i in range(len(data)):  # Для каждой строки данных
        scaled_row = []  # Масштабированная строка
        for j in range(num_features):  # Для каждого признака
            if stds[j] > 0:  # Если отклонение не нулевое
                # Масштабируем значение
                # Формула: (x - среднее) / стандартное отклонение
                scaled_val = (data[i][j] - means[j]) / stds[j]
            else:
                scaled_val = 0.0  # Если все значения одинаковы
            scaled_row.append(scaled_val)  # Добавляем масштабированное значение
        scaled_data.append(scaled_row)  # Добавляем строку в результат
    
    return scaled_data  # Возвращаем масштабированные данные

**Алгоритм кластеризации:**
1. Сначала определяем объем данных в *find_data_volume*.
2. Потом втыкаем туда k штук центров в *generate_random_centers*.
3. Затем группируем данные вокруг этих центров в *assign_points_to_clusters*.
4. Теперь делаем первый шаг, новый набор центров в функции *calculate_new_centers* (то есть считаем центры получившихся кластеров).
5. Повторяем эти операции, пока центры не совпадут — новые со старыми.

In [3]:
# Функция вычисляет евклидово расстояние между двумя точками
def distance(p1, p2):
    # p1 и p2 - списки координат
    sum_sq = 0.0  # Сумма квадратов разностей
    for i in range(len(p1)):  # Для каждой координаты
        diff = p1[i] - p2[i]  # Разность координат
        sum_sq = sum_sq + diff * diff  # Квадрат разности
    return math.sqrt(sum_sq)  # Корень из суммы квадратов
    
# Функция определяет параллелепипед, содержащий все данные
def find_data_volume(data):
    # data - список списков (каждый подсписок - точка)
    if len(data) == 0:  # Если данных нет
        return [[], []]
    
    num_features = len(data[0])  # Количество признаков
    min_values = []  # Минимальные значения для каждого признака
    max_values = []  # Максимальные значения для каждого признака
    
    for j in range(num_features):  # Для каждого признака
        current_min = float('inf')  # Начальное минимальное значение
        current_max = float('-inf')  # Начальное максимальное значение
        
        for i in range(len(data)):  # Для каждой точки
            value = data[i][j]  # Значение признака
            if value < current_min:  # Если меньше текущего минимума
                current_min = value  # Обновляем минимум
            if value > current_max:  # Если больше текущего максимума
                current_max = value  # Обновляем максимум
        
        min_values.append(current_min)  # Добавляем минимум для признака
        max_values.append(current_max)  # Добавляем максимум для признака
    
    return [min_values, max_values]  # Возвращаем список минимумов и список максимумов

# Функция генерирует случайные начальные центры кластеров
def generate_random_centers(volume_data, num_clusters):
    # volume_data - список [min_values, max_values], num_clusters - количество центров
    min_values = volume_data[0]  # Минимальные значения
    max_values = volume_data[1]  # Максимальные значения
    num_features = len(min_values)  # Количество признаков
    centers = {}  # Словарь для хранения центров
    
    for cluster_id in range(num_clusters):  # Для каждого центра
        center_point = []  # Создаем пустой список для центра
        for feature_id in range(num_features):  # Для каждого признака
            # Генерируем случайное число в диапазоне [min, max]
            random_value = random.uniform(min_values[feature_id], max_values[feature_id])
            center_point.append(random_value)  # Добавляем в центр
        centers[cluster_id] = center_point  # Сохраняем центр под ключом cluster_id
    
    return centers  # Возвращаем словарь центров

# Функция группирует точки по ближайшим центрам
def assign_points_to_clusters(centers_dict, data_points):
    # centers_dict - словарь центров (формат: {0: [координаты], 1: [координаты], ...})
    # data_points - список всех точек данных (формат: [[коорд1], [коорд2], ...])
    clusters_dict = {}  # Создаем пустой словарь для хранения кластеров
    
    # Инициализируем пустые списки для ВСЕХ кластеров
    if centers_dict:  # Проверяем, что словарь центров не пустой
        max_cluster_id = max(centers_dict.keys())  # Находим максимальный номер кластера
        for cluster_id in range(max_cluster_id + 1):  # Для каждого номера от 0 до максимума
            clusters_dict[cluster_id] = []  # Создаем пустой список для этого кластера
    else:
        return {}  # Если центров нет - возвращаем пустой словарь
    
    for point in data_points:  # Для каждой точки в данных
        min_distance = float('inf')  # Устанавливаем минимальное расстояние как бесконечность
        best_cluster_id = None  # Пока не знаем лучший кластер
        
        for cluster_id, center_point in centers_dict.items():  # Для каждого центра
            if len(center_point) == 0:  # Если центр пустой (нет координат)
                continue  # Пропускаем его
                
            current_distance = distance(point, center_point)  # Считаем расстояние до центра
            if current_distance < min_distance:  # Если расстояние меньше минимального
                min_distance = current_distance  # Обновляем минимальное расстояние
                best_cluster_id = cluster_id  # Запоминаем номер лучшего кластера
        
        if best_cluster_id is not None:  # Если нашли подходящий центр
            clusters_dict[best_cluster_id].append(point)  # Добавляем точку в этот кластер
        elif clusters_dict:  # Если не нашли (все центры пустые), но словарь не пустой
            clusters_dict[0].append(point)  # Добавляем точку в первый кластер
    
    return clusters_dict  # Возвращает: {0: [точки], 1: [точки], ...} - словарь списков точек

# Функция вычисляет новые центры кластеров
def calculate_new_centers(clusters_dict):
    # clusters_dict - словарь кластеров (формат: {0: [точки], 1: [точки], ...})
    new_centers = {}  # Создаем пустой словарь для новых центров
    
    for cluster_id, points_in_cluster in clusters_dict.items():  # Для каждого кластера
        if points_in_cluster and len(points_in_cluster) > 0:  # Если кластер не пустой
            num_features = len(points_in_cluster[0])  # Узнаем количество признаков
            new_center = []  # Создаем пустой список для нового центра
            
            for feature_id in range(num_features):  # Для каждого признака
                sum_value = 0.0  # Начальная сумма = 0
                for point in points_in_cluster:  # Для каждой точки в кластере
                    sum_value = sum_value + point[feature_id]  # Суммируем значение признака
                mean_value = sum_value / len(points_in_cluster)  # Делим сумму на количество точек
                new_center.append(mean_value)  # Добавляем среднее в новый центр
            
            new_centers[cluster_id] = new_center  # Сохраняем новый центр
        else:
            new_centers[cluster_id] = []  # Для пустого кластера - пустой центр
    
    return new_centers  # Возвращает: {0: [координаты], 1: [координаты], ...} - словарь центров

# Функция сравнивает два набора центров
def are_centers_different(old_centers, new_centers, precision=2):
    # old_centers, new_centers - словари центров
    # precision - точность сравнения (сколько знаков после запятой учитывать)
    
    if set(old_centers.keys()) != set(new_centers.keys()):  # Если ключи разные
        return True  # Центры разные
    
    for cluster_id in old_centers.keys():  # Для каждого кластера
        old_center = old_centers[cluster_id]  # Берем старый центр
        new_center = new_centers[cluster_id]  # Берем новый центр
        
        if len(old_center) != len(new_center):  # Если длины разные
            return True  # Центры разные
        
        if len(old_center) == 0:  # Если оба центра пустые
            continue  # Считаем их одинаковыми
        
        for i in range(len(old_center)):  # Для каждой координаты
            # Сравниваем с заданной точностью
            if round(old_center[i], precision) != round(new_center[i], precision):
                return True  # Нашли различие
    
    return False  # Если не нашли различий - центры одинаковые

# Основная функция кластеризации методом k-средних
def perform_kmeans_clustering(data_points, num_clusters, max_iterations=100):
    # data_points - список точек для кластеризации
    # num_clusters - сколько кластеров нужно получить
    # max_iterations - максимальное количество итераций
    
    data_volume = find_data_volume(data_points)  # Находим границы данных
    current_centers = generate_random_centers(data_volume, num_clusters)  # Случайные начальные центры
    
    iteration = 0  # Счетчик итераций
    while iteration < max_iterations:  # Пока не превысили максимум
        iteration += 1  # Увеличиваем счетчик
        
        current_clusters = assign_points_to_clusters(current_centers, data_points)  # Группируем точки
        new_centers = calculate_new_centers(current_clusters)  # Вычисляем новые центры
        
        if not are_centers_different(current_centers, new_centers):  # Если центры не изменились
            break  # Выходим из цикла - алгоритм сошелся
        
        current_centers = new_centers  # Обновляем центры для следующей итерации
    
    final_clusters = assign_points_to_clusters(current_centers, data_points)  # Финальная группировка
    
    return current_centers, final_clusters  # Возвращает: (центры, кластеры) - кортеж из двух словарей

**Выбор числа кластеров** мы можем делать, перебирая число кластеров от двух и далее (2 , 4, 6, 8) до тех пор, пока квадратичное отклонение не станет практически постоянным. Квадратичное отклонение берется по центрам кластеров. То есть берем отклонение между входящей точкой и центром. Другими словами, это расстояние, возведенное в квадрат и просуммированное.

In [4]:
# Функция вычисляет сумму квадратов ошибок для заданного числа кластеров
def calculate_squared_errors(data_points, num_clusters):
    # data_points - список всех точек данных (вин)
    # num_clusters - количество кластеров k, для которого вычисляем ошибку
    
    best_error = float('inf')  # Начинаем с "бесконечной" ошибки 
    
    for attempt in range(5): # Делаем 5 попыток и берем лучшую (самую маленькую ошибку)
        try:  # Пытаемся выполнить код (защита от возможных ошибок)
            # Выполняем K-means кластеризацию для заданного количества кластеров
            centers, clusters = perform_kmeans_clustering(data_points, num_clusters)
            # centers - словарь центров кластеров {0: [координаты], 1: [координаты], ...}
            # clusters - словарь кластеров {0: [точки], 1: [точки], ...}
            
            total_error = 0.0  # Инициализируем общую ошибку для этой попытки
            
            # Для каждого кластера в результате кластеризации
            for cluster_id, cluster_points in clusters.items():
                # Проверяем три условия:
                # 1. cluster_id есть в словаре centers (центр существует)
                # 2. centers[cluster_id] не пустой список (центр не пустой)
                # 3. cluster_points не пустой список (в кластере есть точки)
                if cluster_id in centers and centers[cluster_id] and cluster_points:
                    center = centers[cluster_id]  # Получаем центр этого кластера
                    
                    # Для каждой точки в этом кластере
                    for point in cluster_points:
                        # Вычисляем квадрат расстояния от точки до центра ее кластера
                        dist_sq = 0.0  # Инициализируем квадрат расстояния
                        
                        # Для каждой координаты точки (признака вина)
                        for i in range(len(point)):
                            diff = point[i] - center[i]  # Разность между точкой и центром
                            dist_sq += diff * diff  # Квадрат разности, добавляем к сумме
                        
                        total_error += dist_sq  # Добавляем ошибку этой точки к общей ошибке
            
            # Если ошибка в этой попытке меньше лучшей найденной
            if total_error < best_error:
                best_error = total_error  # Обновляем лучшую ошибку
        
        except:  # Если произошла ошибка (например, пустые кластеры)
            continue  # Пропускаем эту попытку, пробуем следующую
    
    return best_error  # Возвращаем наименьшую ошибку из 5 попыток

# Функция находит оптимальное количество кластеров
def determine_optimal_clusters(data_points, max_clusters=10):
    # data_points - данные, max_clusters - максимальное количество кластеров для проверки
    error_values = []  # Список для хранения ошибок
    
    print("Вычисление ошибок для разного количества кластеров:")
    for k_value in range(1, max_clusters + 1):  # Для каждого k от 1 до max_clusters
        best_error_value = float('inf')  # Лучшая ошибка (начинаем с бесконечности)
        
        # Несколько попыток для каждого k (из-за случайной инициализации)
        for attempt_number in range(3):
            # Вычисляем квадратичное отклонение для k кластеров
            current_error = calculate_squared_errors(data_points, k_value)  # Вычисляем ошибку
            if current_error < best_error_value:  # Если ошибка лучше текущей лучшей
                best_error_value = current_error  # Обновляем лучшую ошибку
        
        error_values.append(best_error_value)  # Запоминаем ошибку для этого k
        print(f"Количество кластеров={k_value}, ошибка={best_error_value:.2f}")  # Выводим информацию
    
    # Находим оптимальное k (где ошибка перестает значительно уменьшаться)
    optimal_clusters = 2  # Начальное предположение
    max_error_reduction = 0  # Максимальное уменьшение ошибки
    
    for i in range(1, len(error_values) - 1):
        error_reduction = error_values[i-1] - error_values[i]  # Уменьшение ошибки при увеличении k
        if error_reduction > max_error_reduction:  # Если уменьшение больше максимального
            max_error_reduction = error_reduction  # Обновляем максимальное уменьшение
            optimal_clusters = i + 1  # Обновляем оптимальное k
    
    print(f"\nОптимальное количество кластеров: {optimal_clusters}")
    return optimal_clusters, error_values  # Возвращаем оптимальное k и список ошибок

**Восходящая кластеризация.**

Число кластеров заранее неизвестно. Кластеры будем строить и укрупнять их, поэтому она восходящая. Изначально все точки данных, каждая из них будет сама себе кластером. Расстояние между кластерами меньше размера. Если увеличить размер, то количество кластеров будет меньше. Размер говорит о том, когда нужно остановить кластеризацию. Когда расстояние между кластерами превысит указанный размер, нужно остановиться и выдать кластеры.

Алгоритм:
1. Создать 𝑘=𝑁 кластеров, где 𝑁 – число точек в данных.
2. Пока 𝑘>2, найти два ближайших кластера и объединить.
3. Если полученная кластеризация не удовлетворяет требованиям аналитика, то отменить последний шаг 2.

In [5]:
# Функция объединяет два кластера
def merge_two_clusters(cluster_list, first_index, second_index):
    # cluster_list - список кластеров, first_index и second_index - индексы кластеров для объединения
    # Проверяем, что индексы корректны
    if first_index >= second_index:
        # Меняем местами, если first_index не меньше second_index
        first_index, second_index = second_index, first_index
    
    # Объединяем точки двух кластеров
    merged_cluster_points = cluster_list[first_index] + cluster_list[second_index]
    cluster_list[first_index] = merged_cluster_points  # Заменяем первый кластер на объединенный
    
    # Удаляем второй кластер
    del cluster_list[second_index]
    
    return cluster_list  # Возвращаем обновленный список кластеров

# Функция вычисляет евклидово расстояние между двумя точками
def compute_euclidean_distance(first_point, second_point):
    # first_point и second_point - списки координат
    squared_sum = 0.0  # Сумма квадратов разностей
    for coordinate_index in range(len(first_point)):  # Для каждой координаты
        coordinate_diff = first_point[coordinate_index] - second_point[coordinate_index]  # Разность координат
        squared_sum = squared_sum + coordinate_diff * coordinate_diff  # Квадрат разности
    return math.sqrt(squared_sum)  # Корень из суммы квадратов

# Функция вычисляет центр кластера
def compute_cluster_center(cluster_points):
    # cluster_points - список точек кластера
    if len(cluster_points) == 0:  # Если кластер пустой
        return None
    
    num_coordinates = len(cluster_points[0])  # Количество координат
    center_point = []  # Список для центра
    
    for coord_index in range(num_coordinates):  # Для каждой координаты
        coordinate_sum = 0.0  # Сумма значений
        for point in cluster_points:  # Для каждой точки в кластере
            coordinate_sum = coordinate_sum + point[coord_index]  # Суммируем координату
        coordinate_mean = coordinate_sum / len(cluster_points)  # Среднее значение
        center_point.append(coordinate_mean)  # Добавляем в центр
    
    return center_point  # Возвращаем центр кластера

# Функция вычисляет расстояние между двумя кластерами
def calculate_cluster_distance(first_cluster, second_cluster, distance_function):
    # first_cluster и second_cluster - списки точек кластеров
    # distance_function - функция для вычисления расстояния между точками
    first_center = compute_cluster_center(first_cluster)  # Центр первого кластера
    second_center = compute_cluster_center(second_cluster)  # Центр второго кластера
    
    # Проверяем, что оба центра не пустые
    if first_center is None or second_center is None:
        return float('inf')  # Возвращаем большое расстояние
    
    return distance_function(first_center, second_center)  # Расстояние между центрами

# Функция выполняет восходящую кластеризацию
def perform_hierarchical_clustering(data_points, distance_function, size_threshold=1.0):
    """
    Алгоритм восходящей кластеризации.
    Останавливается, когда расстояние между ближайшими кластерами превышает size_threshold.
    """
    
    # Инициализация: каждая точка - отдельный кластер
    clusters_list = []  # Список для хранения кластеров
    for point in data_points:  # Для каждой точки в данных
        clusters_list.append([point])  # Создаем отдельный кластер
    
    # Основной цикл алгоритма
    while True:  # Работаем, пока не выполнится условие остановки
        
        # Вычисление расстояний между всеми парами кластеров
        distance_dict = {}  # Словарь для расстояний (ключ: "i,j", значение: расстояние)
        
        for i in range(len(clusters_list) - 1):  # Для каждого кластера, кроме последнего
            for j in range(i + 1, len(clusters_list)):  # Для всех следующих кластеров
                # Вычисляем расстояние между кластерами i и j
                dist = calculate_cluster_distance(
                    clusters_list[i],  # Первый кластер
                    clusters_list[j],  # Второй кластер
                    distance_function  # Функция для расчета расстояния
                )
                # Сохраняем расстояние с ключом "i,j"
                distance_dict[f"{i},{j}"] = dist
        
        # Поиск двух ближайших кластеров
        min_distance = float('inf')  # Начинаем с максимально возможного расстояния
        best_pair_key = ""  # Ключ для лучшей пары
        
        for key, dist in distance_dict.items():  # Для каждой пары в словаре
            if dist < min_distance:  # Если расстояние меньше текущего минимума
                min_distance = dist  # Обновляем минимум
                best_pair_key = key  # Запоминаем ключ этой пары
        
        # Проверка критерия остановки (по теории)
        if min_distance > size_threshold:  # Если расстояние превышает порог
            print(f"Стоп: расстояние {min_distance:.3f} > порога {size_threshold}")
            break  # Выходим из цикла - алгоритм завершен
        
        # Если условие не выполнено - объединяем кластеры
        # Разбираем ключ на индексы
        idx_parts = best_pair_key.split(',')  # Разделяем строку по запятой
        idx1 = int(idx_parts[0])  # Первый индекс
        idx2 = int(idx_parts[1])  # Второй индекс
        
        print(f"Объединяем {idx1} и {idx2}, расстояние = {min_distance:.3f}")
        
        # Объединяем два ближайших кластера
        clusters_list = merge_two_clusters(clusters_list, idx1, idx2)
        
        # Если остался только 1 кластер - останавливаемся
        if len(clusters_list) == 1:
            print("Остался 1 кластер")
            break
    
    # Преобразуем результат в словарь для удобства
    final_clusters = {}  # Итоговый словарь кластеров
    for i, cluster in enumerate(clusters_list):  # Для каждого кластера
        final_clusters[i] = cluster  # Добавляем в словарь
    
    # Возвращаем кластеры и их количество
    return final_clusters, len(clusters_list)

**Метрика качества - сумма квадратов ошибок (SSE).**

K-means SSE: Сумма квадратов расстояний от точек до известных центров кластеров.

Восходящая SSE: Сумма квадратов расстояний от точек до вычисленных центров кластеров.

Формула идентична. **SSE = СУММА( (точка - центр_её_кластера)² ) для всех точек.**

In [6]:
# Функция сравнивает результаты двух методов кластеризации
def compare_clustering_methods(data_points, kmeans_centers_dict, kmeans_clusters_dict, hierarchical_clusters_dict):
    # data_points - исходные данные (список всех точек)
    # kmeans_centers_dict - словарь центров кластеров от K-means {id: [координаты_центра]}
    # kmeans_clusters_dict - словарь распределения точек по кластерам от K-means {id: [список_точек]}
    # hierarchical_clusters_dict - словарь кластеров от иерархической кластеризации {id: [список_точек]}
    
    # Вычисляем SSE для K-means
    kmeans_sse_value = 0.0  # Инициализируем SSE для K-means нулем
    
    for cluster_id in kmeans_clusters_dict.keys():  # Проходим по всем кластерам K-means
        cluster_points = kmeans_clusters_dict[cluster_id]  # Получаем точки текущего кластера
        cluster_center = kmeans_centers_dict[cluster_id]  # Получаем центр этого кластера
        
        # Проверяем, что центр не пустой (на случай ошибок в алгоритме)
        if len(cluster_center) == 0:
            continue  # Пропускаем этот кластер, если центр пустой
            
        for point in cluster_points:  # Для каждой точки в текущем кластере
            point_sse = 0.0  # Инициализируем SSE для текущей точки
            
            # Сумма квадратов разностей координат между точкой и центром кластера
            for coord_index in range(len(point)):  # Для каждой координаты (признака)
                coord_diff = point[coord_index] - cluster_center[coord_index]  # Разность координаты точки и центра
                point_sse = point_sse + coord_diff * coord_diff  # Квадрат разности добавляем к SSE точки
            
            kmeans_sse_value = kmeans_sse_value + point_sse  # Добавляем SSE точки к общей SSE K-means
    
    # Вычисляем SSE для иерархической кластеризации
    hierarchical_sse_value = 0.0  # Инициализируем SSE для иерархической кластеризации нулем
    
    for cluster_id in hierarchical_clusters_dict.keys():  # Проходим по всем кластерам иерархической кластеризации
        cluster_points = hierarchical_clusters_dict[cluster_id]  # Получаем точки текущего кластера
        cluster_center = compute_cluster_center(cluster_points)  # Вычисляем центр кластера (среднее всех точек)
        
        # Проверяем, что центр удалось вычислить (кластер не пустой)
        if cluster_center is None:
            continue  # Пропускаем этот кластер, если центр не вычислен
            
        for point in cluster_points:  # Для каждой точки в текущем кластере
            point_sse = 0.0  # Инициализируем SSE для текущей точки
            
            # Сумма квадратов разностей координат между точкой и центром кластера
            for coord_index in range(len(point)):  # Для каждой координаты (признака)
                coord_diff = point[coord_index] - cluster_center[coord_index]  # Разность координаты точки и центра
                point_sse = point_sse + coord_diff * coord_diff  # Квадрат разности добавляем к SSE точки
            
            hierarchical_sse_value = hierarchical_sse_value + point_sse  # Добавляем SSE точки к общей SSE иерархической
    
    return kmeans_sse_value, hierarchical_sse_value  # Возвращаем кортеж: (SSE_K-means, SSE_иерархическая)

In [7]:
# Основная программа
if __name__ == "__main__":
    # Загрузка и подготовка данных
    print("=" * 60)
    print("ЗАГРУЗКА ДАННЫХ")
    print("=" * 60)
    original_data, column_headers = load_data('WineQT.csv')  # Загружаем данные из CSV файла
    normalized_data = scale_data(original_data)  # Нормализуем данные (z-score нормализация)
    
    # Определяем оптимальное число кластеров методом локтя
    print("\n" + "=" * 60)
    print("ОПРЕДЕЛЕНИЕ ОПТИМАЛЬНОГО ЧИСЛА КЛАСТЕРОВ")
    print("=" * 60)
    optimal_cluster_count, error_list = determine_optimal_clusters(normalized_data, max_clusters=10)  # Находим оптимальное k
    
    # K-means кластеризация
    print("\n" + "=" * 60)
    print(f"K-MEANS КЛАСТЕРИЗАЦИЯ ({optimal_cluster_count} КЛАСТЕРОВ)")
    print("=" * 60)
    kmeans_final_centers, kmeans_final_clusters = perform_kmeans_clustering(normalized_data, optimal_cluster_count)  # Выполняем K-means
    
    # Восходящая кластеризация (на подвыборке для скорости)
    print("\n" + "=" * 60)
    print("ВОСХОДЯЩАЯ КЛАСТЕРИЗАЦИЯ")
    print("=" * 60)
    sample_data_size = min(200, len(normalized_data))  # Ограничиваем размер выборки для скорости
    data_sample = normalized_data[:sample_data_size]  # Берем подвыборку
    print(f"Используем подвыборку из {sample_data_size} точек")
    print(f"Порог остановки (size_threshold): 2.0")
    
    # Выполняем иерархическую кластеризацию с критерием остановки по расстоянию
    hierarchical_final_clusters, num_clusters_hierarchical = perform_hierarchical_clustering(
        data_sample, 
        compute_euclidean_distance, 
        size_threshold=2.0  # Критерий остановки: когда расстояние > 2.0
    )
    
    print(f"\nПолучено кластеров: {num_clusters_hierarchical}")
    
    # Сравнение методов кластеризации
    print("\n" + "=" * 60)
    print("СРАВНЕНИЕ МЕТОДОВ КЛАСТЕРИЗАЦИИ")
    print("=" * 60)
    
    # Подготовка данных для сравнения K-means (только для подвыборки)
    kmeans_sample_clusters = {}  # Словарь для хранения кластеров K-means на подвыборке
    for cluster_id in kmeans_final_clusters.keys():  # Инициализируем пустые кластеры
        kmeans_sample_clusters[cluster_id] = []
    
    # Распределяем точки подвыборки по кластерам K-means
    for data_point in data_sample:  # Для каждой точки в подвыборке
        min_distance_found = float('inf')  # Инициализируем минимальное расстояние
        best_cluster_id_found = 0  # ID лучшего кластера
        
        # Ищем ближайший центр кластера
        for cluster_id in kmeans_final_centers.keys():  # Для каждого центра K-means
            current_distance = distance(data_point, kmeans_final_centers[cluster_id])  # Считаем расстояние
            if current_distance < min_distance_found:  # Если расстояние меньше
                min_distance_found = current_distance  # Обновляем минимум
                best_cluster_id_found = cluster_id  # Запоминаем лучший кластер
        
        # Добавляем точку в ближайший кластер
        kmeans_sample_clusters[best_cluster_id_found].append(data_point)
    
    # Вычисляем SSE (сумму квадратов ошибок) для K-means
    kmeans_sample_sse = 0.0  # Инициализируем SSE для K-means
    
    for cluster_id in kmeans_sample_clusters.keys():  # Для каждого кластера K-means
        cluster_points = kmeans_sample_clusters[cluster_id]  # Точки кластера
        if len(cluster_points) > 0:  # Если кластер не пустой
            cluster_center_point = kmeans_final_centers[cluster_id]  # Центр кластера
            
            if len(cluster_center_point) == 0:  # Если центр пустой
                continue  # Пропускаем
                
            # Считаем SSE для всех точек кластера
            for point in cluster_points:  # Для каждой точки
                point_sse = 0.0  # SSE для точки
                for coord_index in range(len(point)):  # Для каждой координаты
                    # Разность между точкой и центром
                    coord_diff = point[coord_index] - cluster_center_point[coord_index]
                    point_sse = point_sse + coord_diff * coord_diff  # Квадрат разности
                kmeans_sample_sse = kmeans_sample_sse + point_sse  # Добавляем к общей SSE
    
    # Вычисляем SSE для иерархической кластеризации
    hierarchical_sse_result = 0.0  # Инициализируем SSE для иерархической кластеризации
    
    for cluster_id in hierarchical_final_clusters.keys():  # Для каждого кластера
        cluster_points = hierarchical_final_clusters[cluster_id]  # Точки кластера
        if len(cluster_points) > 0:  # Если кластер не пустой
            cluster_center_point = compute_cluster_center(cluster_points)  # Вычисляем центр кластера
            
            if cluster_center_point is None:  # Если центр не удалось вычислить
                continue  # Пропускаем
                
            # Считаем SSE для всех точек кластера
            for point in cluster_points:  # Для каждой точки
                point_sse = 0.0  # SSE для точки
                for coord_index in range(len(point)):  # Для каждой координаты
                    # Разность между точкой и центром
                    coord_diff = point[coord_index] - cluster_center_point[coord_index]
                    point_sse = point_sse + coord_diff * coord_diff  # Квадрат разности
                hierarchical_sse_result = hierarchical_sse_result + point_sse  # Добавляем к общей SSE
    
    # Вывод результатов сравнения
    print("\nМЕТРИКИ КАЧЕСТВА (SSE - сумма квадратов ошибок):")
    print("-" * 40)
    print(f"K-means SSE:          {kmeans_sample_sse:>10.4f}")
    print(f"Иерархическая SSE:    {hierarchical_sse_result:>10.4f}")
    print(f"Разница:              {abs(kmeans_sample_sse - hierarchical_sse_result):>10.4f}")
    
    # Анализ размеров кластеров
    print("\nРАЗМЕРЫ КЛАСТЕРОВ:")
    print("-" * 40)
    
    # Размеры кластеров K-means
    kmeans_cluster_sizes_list = []
    for cluster_id in kmeans_sample_clusters.keys():
        kmeans_cluster_sizes_list.append(len(kmeans_sample_clusters[cluster_id]))
    
    # Размеры кластеров иерархической
    hierarchical_cluster_sizes_list = []
    for cluster_id in hierarchical_final_clusters.keys():
        hierarchical_cluster_sizes_list.append(len(hierarchical_final_clusters[cluster_id]))
    
    print(f"K-means ({len(kmeans_cluster_sizes_list)} кластеров):")
    print(f"  Размеры: {kmeans_cluster_sizes_list}")
    print(f"  Мин размер: {min(kmeans_cluster_sizes_list)}")
    print(f"  Макс размер: {max(kmeans_cluster_sizes_list)}")
    print(f"  Средний размер: {sum(kmeans_cluster_sizes_list)/len(kmeans_cluster_sizes_list):.1f}")
    
    print(f"\nИерархическая ({len(hierarchical_cluster_sizes_list)} кластеров):")
    print(f"  Мин размер: {min(hierarchical_cluster_sizes_list)}")
    print(f"  Макс размер: {max(hierarchical_cluster_sizes_list)}")
    print(f"  Средний размер: {sum(hierarchical_cluster_sizes_list)/len(hierarchical_cluster_sizes_list):.1f}")
    
    # Статистика по малым кластерам (меньше 5 точек)
    small_clusters_hierarchical = sum(1 for size in hierarchical_cluster_sizes_list if size <= 3)
    print(f"  Кластеров с ≤3 точками: {small_clusters_hierarchical} ({small_clusters_hierarchical/len(hierarchical_cluster_sizes_list)*100:.1f}%)")
    
    # Примеры кластеров (вывод в компактном формате)
    print("\n" + "=" * 60)
    print("ПРИМЕРЫ КЛАСТЕРОВ (первые 3 признака)")
    print("=" * 60)
    
    print("\nK-MEANS КЛАСТЕРЫ:")
    print("-" * 40)
    for cluster_id in kmeans_sample_clusters.keys():
        size = len(kmeans_sample_clusters[cluster_id])
        if size > 0:
            # Берем первую точку для примера, показываем 3 признака
            point = kmeans_sample_clusters[cluster_id][0]
            short_point = [round(coord, 3) for coord in point[:3]]  # Округляем для читаемости
            print(f"Кластер {cluster_id}: {size:3d} точек | Пример: {short_point}")
    
    print("\nИЕРАРХИЧЕСКИЕ КЛАСТЕРЫ (первые 10 крупнейших):")
    print("-" * 40)
    
    # Сортируем кластеры по размеру (убывание)
    sorted_clusters = sorted(
        [(cluster_id, len(points)) for cluster_id, points in hierarchical_final_clusters.items()],
        key=lambda x: x[1],
        reverse=True
    )
    
    # Выводим топ-10 кластеров
    for i in range(min(10, len(sorted_clusters))):
        cluster_id, size = sorted_clusters[i]
        if size > 0:
            point = hierarchical_final_clusters[cluster_id][0]
            short_point = [round(coord, 3) for coord in point[:3]]
            print(f"Кластер {cluster_id:3d}: {size:3d} точек | Пример: {short_point}")
    
    # Финальные выводы
    print("\n" + "=" * 60)
    print("ВЫВОДЫ")
    print("=" * 60)
    
    print("\n1. КРИТЕРИЙ ОСТАНОВКИ ИЕРАРХИЧЕСКОЙ КЛАСТЕРИЗАЦИИ:")
    print("   • Начинаем с N кластеров (каждая точка — отдельный кластер)")
    print("   • На каждом шаге объединяем два ближайших кластера")
    print("   • Останавливаемся, когда расстояние между ближайшими кластерами")
    print(f"     превышает заданный порог (size_threshold = {2.0})")
    
    print("\n2. СРАВНЕНИЕ РЕЗУЛЬТАТОВ:")
    print(f"   • K-means создал {len(kmeans_cluster_sizes_list)} кластера")
    print(f"   • Иерархическая кластеризация создала {len(hierarchical_cluster_sizes_list)} кластеров")
    
    if hierarchical_sse_result < kmeans_sample_sse:
        print("   • Иерархическая кластеризация дала меньшую SSE (более плотные кластеры)")
    else:
        print("   • K-means дал меньшую SSE (более компактные кластеры)")

ЗАГРУЗКА ДАННЫХ

ОПРЕДЕЛЕНИЕ ОПТИМАЛЬНОГО ЧИСЛА КЛАСТЕРОВ
Вычисление ошибок для разного количества кластеров:
Количество кластеров=1, ошибка=12573.00
Количество кластеров=2, ошибка=10238.47
Количество кластеров=3, ошибка=9105.26
Количество кластеров=4, ошибка=8087.14
Количество кластеров=5, ошибка=7256.54
Количество кластеров=6, ошибка=7274.56
Количество кластеров=7, ошибка=6472.38
Количество кластеров=8, ошибка=6602.72
Количество кластеров=9, ошибка=5852.26
Количество кластеров=10, ошибка=6135.68

Оптимальное количество кластеров: 2

K-MEANS КЛАСТЕРИЗАЦИЯ (2 КЛАСТЕРОВ)

ВОСХОДЯЩАЯ КЛАСТЕРИЗАЦИЯ
Используем подвыборку из 200 точек
Порог остановки (size_threshold): 2.0
Объединяем 0 и 4, расстояние = 0.000
Объединяем 44 и 45, расстояние = 0.000
Объединяем 57 и 62, расстояние = 0.000
Объединяем 58 и 62, расстояние = 0.000
Объединяем 65 и 67, расстояние = 0.000
Объединяем 67 и 69, расстояние = 0.000
Объединяем 71 и 74, расстояние = 0.000
Объединяем 86 и 88, расстояние = 0.000
Объединяем 88 